![Memories Teach](https://lh3.google.com/u/2/d/11o4JyBYhHcmz-LSt63GpOs4lfW9j39T_=w1912-h954-iv1)


`Basic to Advance in Google colab's for image processing, pattern recognition and computer vision`

[Phonepaserth SISAYKEO]

Reference: visioncolab

# Basic Feature Selection and Transformation

SFS, ExSearch, SelectKBest, SBS, RFECV, PCA, ICA, PLSR

# Libraries

In [ ]:
!git clone https://github.com/domingomery/balu3
!pip install ./balu3

Cloning into 'balu3'...
remote: Enumerating objects: 267, done.
remote: Counting objects: 100% (267/267), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 267 (delta 155), reused 181 (delta 75), pack-reused 0
Receiving objects: 100% (267/267), 70.23 KiB | 8.78 MiB/s, done.
Resolving deltas: 100% (155/155), done.
Processing ./balu3
  Preparing metadata (setup.py) ... done
  Created wheel for balu3: filename=balu3-1.0-py3-none-any.whl size=43700 sha256=66783b2bc92bab765ce7b71b87d44d68d87a9ffe22c2489fff26f54b607306d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-k1s_khj7/wheels/9e/f6/a9/3d39da83a3504331129d706f5b04ceb3dbf0ec0f313cdb8fcf
Successfully built balu3


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from   sklearn.metrics import confusion_matrix, accuracy_score
from   seaborn import heatmap
from   tqdm.auto import tqdm

from   balu3.fs.sel    import sfs,clean,exsearch
from   balu3.ft.trans  import pca
from   balu3.cl.basics import SplitTrainTest
from   balu3.cl.basics import ClassifierKNN
from   balu3.io.misc   import imageload
from   balu3.ft.norm   import minmax

## Database loading

In this example, we will use the features already extracted in the example that performs the recognition of characters A,B,X,Y,Z with different sizes and different orientations. The descriptors are used for the solution since they are rotation and scale invariant.

The database consists of 5 classes and 100 images per class.

In [ ]:
!wget https://www.dropbox.com/s/5sj3y39pm9uuj8w/Features_Letras.zip
!unzip -qq Features_Letras

--2023-07-18 19:34:49--  https://www.dropbox.com/s/5sj3y39pm9uuj8w/Features_Letras.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/5sj3y39pm9uuj8w/Features_Letras.zip [following]
--2023-07-18 19:34:49--  https://www.dropbox.com/s/raw/5sj3y39pm9uuj8w/Features_Letras.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc43ac9f9d41a6a3c741228359c8.dl.dropboxusercontent.com/cd/0/inline/CAEe-oLvkruB02mifFEbTyxbLtdpI6DBZk2gXPBQl_yynmo9Ouam8JX16jqvEIa3Auzu4mlfzS3jEjGkJCJbzDhCp5xjUylBQeHyxuW09byDwfKmCClOGlFrgma8J_AP1Zx8gzLJkaeXKJff1MAIITIG_Tm781EnLL5fa_W2qtlZFA/file# [following]
--2023-07-18 19:34:49--  https://uc43ac9f9d41a6a3c741228359c8.dl.dropboxusercontent.com/cd/0/inline/CAEe-oLvkruB02mifFEbTyxbLtdpI6DBZk2gXPBQl_yynmo9Ouam8JX16jq

## Functions

In [ ]:
def load_features(prefix):
    X = np.load(prefix+'_X.npy')            # features
    y = np.load(prefix+'_y.npy')            # labels
    print('Features: '+str(X.shape[0]) +' samples with '+str(X.shape[1]) +' features')
    print(' Classes: '+str(int(np.min(y)))+'...'+str(int(np.max(y))))
    return X,y

# 0) Definitions

In [ ]:
# 0. KNN
neighbors = 1
normalizar = False

# 1. Features
X,y = load_features('Features_Letras/Letras')
y = y.astype(int) #conversión de float a enteros

# 2. Split Training/Testing
ntrain = 75
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain)

# Classification
print()
print('Evaluation with Original Data:')
print('================================')

acc,C = ClassifierKNN(Xtrain,ytrain,Xtest,ytest,n_neighbors=neighbors,normalize=normalizar)
# 3. Cleaning
sclean = clean(Xtrain)            # indices of selected features
Xtrain_clean = Xtrain[:,sclean]   # new training
Xtest_clean  = Xtest[:,sclean]    # new testing
print()
print('Cleaning:')
print('=========')
print('Selection: '+str(len(sclean))+'/'+str(X.shape[1])+' features: '+str(sclean))

# 4. Normalization
Xtrain_norm, a, b = minmax(Xtrain_clean)
Xtest_norm        = Xtest_clean * a + b

# Classification with normalized data
print()
print('Evaluation with cleaned and normalized Data:')
print('=======================================================')
acc,C = ClassifierKNN(Xtrain_norm,ytrain,Xtest_norm,ytest,n_neighbors=neighbors,normalize=normalizar)



Features: 500 samples with 45 features
 Classes: 0...4

Evaluation with Original Data:
Training with 375 samples. Testing with 125 samples (of 45 features)...
Testing Accuracy = 70.39999999999999%
Confusion Matrix:
[[17  2  1  1  4]
 [ 0 20  4  1  0]
 [ 1  2 18  0  4]
 [ 6  0  0 14  5]
 [ 0  4  1  1 19]]

Cleaning:
Selection: 41/45 features: [ 0  1  2  3  4  5  8  9 10 11 12 13 15 16 17 18 19 20 21 22 23 24 25 26
 27 28 29 30 31 32 33 34 35 36 37 38 39 41 42 43 44]

Evaluation with cleaned and normalized Data:
Training with 375 samples. Testing with 125 samples (of 41 features)...
Testing Accuracy = 79.2%
Confusion Matrix:
[[19  0  2  4  0]
 [ 0 25  0  0  0]
 [ 5  0 20  0  0]
 [ 5  0  0 16  4]
 [ 0  0  0  6 19]]


# 1) Feature Selection

## SFS (Library Balu3)

In [ ]:
# 5. SFS - Fisher
sel = sfs(Xtrain_norm,ytrain,8,show=True)
Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]

# 6, 7, 8. Classification using selected features
print()
print('SFS results:')
print('==============')
acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)



Selecting Features: 100%|██████████| 8.00/8.00 [00:00<00:00, 43.2 features/s]


SFS results:
Training with 375 samples. Testing with 125 samples (of 8 features)...
Testing Accuracy = 95.19999999999999%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


## Exhaustive Search (Library Balu3)

In [ ]:
# 5. Selection

# First we make a pre-selection to narrow down the search space
sel_1 = sfs(Xtrain_norm,ytrain,16,show=True)
Xtrain_sel_1 = Xtrain_norm[:,sel_1]
Xtest_sel_1  = Xtest_norm[:,sel_1]

# Then we do an exhaustive search for
sel_2 = exsearch(Xtrain_sel,ytrain,6,show=True)
Xtrain_sel_2 = Xtrain_sel_1[:,sel_2]
Xtest_sel_2  = Xtest_sel_1[:,sel_2]


# 6, 7, 8. Clasification and evaluation
print()
print('Exhaustive Search Results:')
print('==========================')
acc,C = ClassifierKNN(Xtrain_sel_2,ytrain,Xtest_sel_2,ytest,n_neighbors=neighbors,normalize=normalizar)


Selecting Features: 100%|██████████| 16.0/16.0 [00:00<00:00, 17.0 features/s]
Combinations checked: 100%|██████████| 28.0/28.0 [00:00<00:00, 459 combinations/s]


Exhaustive Search Results:
Training with 375 samples. Testing with 125 samples (of 6 features)...
Testing Accuracy = 92.0%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 21  0  4  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


## RFECV (Library sklearn)

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC

estimator = SVC(kernel="linear")


selector  = RFECV(estimator, step=1,cv=5)
selector  = selector.fit(Xtrain_norm, ytrain)
sel       = np.nonzero(selector.support_)[0]

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]
acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


Training with 375 samples. Testing with 125 samples (of 12 features)...
Testing Accuracy = 95.19999999999999%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


## RFE (Library sklearn)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

estimator = SVC(kernel="linear")

p = 8
selector  = RFE(estimator)
selector  = selector.fit(Xtrain_norm, ytrain)
sel       = np.nonzero(selector.support_)[0]

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]
acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


Training with 375 samples. Testing with 125 samples (of 20 features)...
Testing Accuracy = 84.0%
Confusion Matrix:
[[23  0  2  0  0]
 [ 0 25  0  0  0]
 [ 6  0 19  0  0]
 [ 6  0  0 19  0]
 [ 0  0  0  6 19]]


## SFS (Library mlxtend)

In [ ]:
import joblib
import sys
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as mlxsfs
from   sklearn.neighbors import KNeighborsClassifier

In [ ]:
estimator = KNeighborsClassifier(n_neighbors=5)
p = 12
forward = True
sfsx = mlxsfs(estimator, k_features=p,forward=forward,floating=False,verbose=2,scoring='accuracy',cv=10)

sfsx = sfsx.fit(Xtrain_norm, ytrain)
sel = list(sfsx.k_feature_idx_)

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]
acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.9s

[2023-07-18 19:37:37] Features: 1/12 -- score: 0.7189900426742531[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.2s

[2023-07-18 19:37:38] Features: 2/12 -- score: 0.9301564722617354
[2023-07-18 19:37:39] Features: 3/12 -- score: 0.972972972972973
[2023-07-18 19:37:40] Features: 4/12 -- score: 0.991891891891892
[2023-07-18 19:37:41] Features: 5/12 -- score: 0.991891891891892
[2023-07-18 19:37:42] Features: 6/12 -- score: 0.991891891891892
[2023-07-18 19:37:42] Features: 7/12 -- score: 0.991891891891892
[2023-07-18 19:37:43] Features: 8/12 -- score: 1.0
[2023-07-18 19:37:44] Features: 9/12 -- score: 1.0
[2023-07-18 19:37:45] Features: 10/12 -- score: 1.0
[2023-07-18 19:37:46] Features: 11/12 -- score: 1.0

Training with 375 samples. Testing with 125 samples (of 12 features)...
Testing Accuracy = 95.19999999999999%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]



[2023-07-18 19:37:46] Features: 12/12 -- score: 1.0

## SBS (Library mlxtend)

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as mlxsfs

estimator = KNeighborsClassifier(n_neighbors=5)
p = 12
forward = False
sbsx = mlxsfs(estimator, k_features=p,forward=forward,floating=False,verbose=2,scoring='accuracy',cv=10)

sbsx = sbsx.fit(Xtrain_norm, ytrain)
sel = list(sbsx.k_feature_idx_)

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]
acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.1s

[2023-07-18 19:37:47] Features: 40/12 -- score: 0.9702702702702704[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    1.1s

[2023-07-18 19:37:48] Features: 39/12 -- score: 0.972972972972973
[2023-07-18 19:37:51] Features: 38/12 -- score: 0.9756756756756756
[2023-07-18 19:37:52] Features: 37/12 -- score: 0.9864864864864865
[2023-07-18 19:37:53] Features: 36/12 -- score: 0.9864864864864865
[2023-07-18 19:37:54] Features: 35/12 -- score: 0.9864864864864865
[2023-07-18 19:37:54] Features: 34/12 -- score: 0.9864864864864865
[2023-07-18 19:37:55] Features: 33/12 -- score: 0.9864864864864865
[2023-07-18 19:37:56] Features: 32/12 -- score: 0.9891891891891891
[2023-07-18 19:37:57] Features: 31/12 -- score: 0.9972972972972973
[2023-07-18 19:37:58] Features: 30/12 -- score: 0.9972972972972973
[2023-07-18 19:37:59] Features: 29/12 -- score: 0.9972972972972973
[2023-07-18 1

Training with 375 samples. Testing with 125 samples (of 12 features)...
Testing Accuracy = 91.2%
Confusion Matrix:
[[23  0  2  0  0]
 [ 0 22  0  3  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]



[2023-07-18 19:38:09] Features: 12/12 -- score: 0.9945945945945945

## SelectKBest (Library sklearn)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

p = 8
model =  SelectKBest(chi2,k=p)

eps = 1e-8
model.fit(Xtrain_norm+eps, ytrain)

sel        = np.nonzero(model.get_support())[0]

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]

acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


Training with 375 samples. Testing with 125 samples (of 8 features)...
Testing Accuracy = 92.80000000000001%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 22  0  3  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


## Selection based on Trees (Library sklearn)

In [ ]:

from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

p = 10

lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

model = SelectFromModel(lgbc, max_features=p)
model.fit(Xtrain_norm, ytrain)

sel        = np.nonzero(model.get_support())[0]

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]

acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


Training with 375 samples. Testing with 125 samples (of 10 features)...
Testing Accuracy = 74.4%
Confusion Matrix:
[[23  1  0  1  0]
 [ 0 21  0  4  0]
 [11  0 13  0  1]
 [ 4  0  0 17  4]
 [ 0  0  5  1 19]]


In [ ]:

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

p = 8

model = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=p)
model.fit(Xtrain_norm, ytrain)

sel        = np.nonzero(model.get_support())[0]

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]

acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


Training with 375 samples. Testing with 125 samples (of 8 features)...
Testing Accuracy = 92.0%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 21  0  4  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


In [ ]:

from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

p = 10
model = SelectFromModel(LogisticRegression(penalty="l2"), max_features=p)
model.fit(Xtrain_norm, ytrain)

sel        = np.nonzero(model.get_support())[0]

Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]

acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest,n_neighbors=neighbors,normalize=normalizar)


Training with 375 samples. Testing with 125 samples (of 10 features)...
Testing Accuracy = 95.19999999999999%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


# 2) Feature Transformation

## PCA (Library Balu3)

In [ ]:
sel = sfs(Xtrain_norm,ytrain,20)
Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]

Xtrain_pca, _, A, Xm, _ = pca(Xtrain_sel, n_components=10)
Xtest_pca = np.matmul(Xtest_sel - Xm, A) # Paso 5: PCA

acc,C = ClassifierKNN(Xtrain_pca,ytrain,Xtest_pca,ytest,n_neighbors=neighbors,normalize=normalizar)

Training with 375 samples. Testing with 125 samples (of 10 features)...
Testing Accuracy = 89.60000000000001%
Confusion Matrix:
[[24  0  1  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  2  0 19  4]
 [ 0  0  0  6 19]]


## ICA (Library sklearn)

In [ ]:
from sklearn.decomposition import FastICA

sel = sfs(Xtrain_norm,ytrain,30)
Xtrain_sel = Xtrain_norm[:,sel]
Xtest_sel  = Xtest_norm[:,sel]

ica = FastICA(n_components=8, random_state=0)
ica.fit(Xtrain_sel, ytrain)

Xtrain_ica = ica.transform(Xtrain_sel)
Xtest_ica  = ica.transform(Xtest_sel)

acc,C = ClassifierKNN(Xtrain_ica,ytrain,Xtest_ica,ytest,n_neighbors=neighbors,normalize=normalizar)


Training with 375 samples. Testing with 125 samples (of 8 features)...
Testing Accuracy = 91.2%
Confusion Matrix:
[[24  0  0  0  1]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 21  4]
 [ 0  0  0  6 19]]


## PLSR (Library sklearn)

In [ ]:
from sklearn.cross_decomposition import PLSRegression

plsr = PLSRegression(n_components=10)

plsr.fit(Xtrain_norm, ytrain)

Xtrain_plsr = plsr.transform(Xtrain_norm)
Xtest_plsr  = plsr.transform(Xtest_norm)

acc,C = ClassifierKNN(Xtrain_plsr,ytrain,Xtest_plsr,ytest,n_neighbors=neighbors,normalize=normalizar)




Training with 375 samples. Testing with 125 samples (of 10 features)...
Testing Accuracy = 94.39999999999999%
Confusion Matrix:
[[23  1  1  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 4  0  0 21  0]
 [ 0  0  0  1 24]]


# 3) Transformation and Selectionof Features

## PCA and SFS (Library Balu3)

In [ ]:
sel_1 = sfs(Xtrain_norm,ytrain,12)
Xtrain_sel_1 = Xtrain_norm[:,sel_1]
Xtest_sel_1  = Xtest_norm[:,sel_1]

Xtrain_pca, _, A, Xm, _ = pca(Xtrain_sel_1, n_components=4)
Xtest_pca = np.matmul(Xtest_sel_1 - Xm, A) # Paso 5: PCA

Xtrain_sfspca = np.concatenate((Xtrain_sel_1,Xtrain_pca),axis=1)
Xtest_sfspca  = np.concatenate((Xtest_sel_1,Xtest_pca),axis=1)

sel_2 = sfs(Xtrain_sfspca,ytrain,8)
Xtrain_sel_2 = Xtrain_sfspca[:,sel_2]
Xtest_sel_2  = Xtest_sfspca[:,sel_2]


acc,C = ClassifierKNN(Xtrain_sel_2,ytrain,Xtest_sel_2,ytest,n_neighbors=neighbors,normalize=normalizar)

Training with 375 samples. Testing with 125 samples (of 8 features)...
Testing Accuracy = 95.19999999999999%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  0  6 19]]


## ICA and SFS (Libraries sklearn and Balu3)

In [ ]:
sel_1 = sfs(Xtrain_norm,ytrain,12)
Xtrain_sel_1 = Xtrain_norm[:,sel_1]
Xtest_sel_1  = Xtest_norm[:,sel_1]

ica = FastICA(n_components=8, random_state=0)
ica.fit(Xtrain_sel, ytrain)

Xtrain_ica = ica.transform(Xtrain_sel)
Xtest_ica  = ica.transform(Xtest_sel)

Xtrain_sfsica = np.concatenate((Xtrain_sel_1,Xtrain_ica),axis=1)
Xtest_sfsica  = np.concatenate((Xtest_sel_1,Xtest_ica),axis=1)

sel_2 = sfs(Xtrain_sfsica,ytrain,8)
Xtrain_sel_2 = Xtrain_sfsica[:,sel_2]
Xtest_sel_2  = Xtest_sfsica[:,sel_2]


acc,C = ClassifierKNN(Xtrain_sel_2,ytrain,Xtest_sel_2,ytest,n_neighbors=neighbors,normalize=normalizar)

Training with 375 samples. Testing with 125 samples (of 8 features)...
Testing Accuracy = 95.19999999999999%
Confusion Matrix:
[[25  0  0  0  0]
 [ 0 25  0  0  0]
 [ 0  0 25  0  0]
 [ 0  0  0 25  0]
 [ 0  0  6  0 19]]
